## Using SOMA to label unlabeled mocaps

We base this off of the two SOMA tutorials: [Run SOMA on SOMA dataset](https://github.com/nghorbani/soma/blob/main/src/tutorials/run_soma_on_soma_dataset.ipynb) and [Label Priming](https://github.com/nghorbani/soma/blob/main/src/tutorials/label_priming.ipynb).

Reading the [definitions](https://github.com/nghorbani/soma/tree/main/src/tutorials#definitions) of terminology is recommended.

We will be using a pre-trained SOMA model called the "SuperSet" to label our unlabeled motioin capture data. The SuperSet is a pre-trained body model with 89 markers placed around the body, and thus will label our unlabeled mocap data by matching our unlabeled markers to a subset of the 89 markers. If we have an unlabeled marker that does not correspond to a marker on the SuperSet, the behavior is undefined -- either that marker will be unlabeled (marked as ghost points/discarded), or it will be labeled with some nearby marker label. This imperfect labeling can lead to downstream errors when trying to fit the SMPL model to the labeled mocap data.

It is possible to train a new SOMA model, for a different marker layout (specifically a marker layout that does not fit a subset of the SuperSet), please check this [README](https://github.com/nghorbani/soma/blob/main/src/tutorials/README.md#training-soma) and [Tutorial 1](https://github.com/nghorbani/soma/blob/main/src/tutorials/run_soma_on_soma_dataset.ipynb) for information on this. As per the note in Tutorial 1:


>Due to licensing restrictions we cannot release the AMASS marker noise model and the CAESAR beta parameters.
>
>In an ablative study in the paper, we have shown that these parameters improve the performance SOMA. So without them it might be that the model you train would be underperforming
>

Thus it is more effective to use the pretrained models. As such, we will not be covering model training in this tutorial.

Conceptually, this is how the SOMA training works (see [this paper](https://arxiv.org/pdf/2110.04431.pdf) for more details):
- Goal: to label unlabeled mocap data for a specific *markerlayout*
- First, we take a single frame from a single trial of our unlabeled mocap data
  - one frame needed per subject, since each subject has a unique body -- and thus unique markerlayouts/point clouds -- thus a unique body model
- This frame is manually labeled
- This labeled frame is MoSHed to get a SMPL-X body model for that specific subject
- virtual markers corresponding to our markerlayout are automotically placed on this SMPL-X body
- This SMPL body model is animated by AMASS motions
  - AMASS dataset has many actions saved in SMPL format, which allows us to have our subject-specific body-model 'perform' many actions
  - Our markerlayout remains on the body model as these actions are performed
- Thus a large repository of labeled mocap data is synthesized, data which is specific to our subject and our markerlayout
- Real noise data is added to this synthetic data
- occlusions and ghost points also added
- SOMA model is trained on this synthetic data, where the model is given the input of a synthetic unlabeled point cloud and learns to output the corresponding labels
- Transformers with multiple self-attention units are used to learn the spatial structure of the 3d body, as well as an optimal transport layer to encode the natural constraints of the human body.

### Environment Check and Imports

In [1]:
# If this doesn't work, something is wrong with the conda environment. Check conda-env.md
# Make sure you are using the right kernel
import soma
import psbody
import moshpp

In [2]:
import os
import os.path as osp
import time

from loguru import logger

from soma.tools.run_soma_multiple import run_soma_on_multiple_settings


In [ ]:
import inspect
print(inspect.getsourcefile(run_soma_on_multiple_settings))

/home/ritaank/Documents/dev/motion_synthesis/soma-root-2/soma/src/soma/tools/run_soma_multiple.py


### Configuration Files

SOMA uses OmegaConf for configuration of each run. You can change every value of the configuration inside this notebook, so you do not need to change the YAML file, unless you want to change the default value for future cases.

In [ ]:
def get_soma_conf_file():
    '''
    Prints path of soma_run_conf.yaml to inspect exact configuration settings
    '''
    import soma
    init_path = osp.join(soma.__file__)
    base_path = osp.dirname(osp.dirname(osp.dirname(init_path)))
    conf_path = osp.join(base_path, 'support_data', 'conf', 'soma_run_conf.yaml')

    return conf_path

In [ ]:
## View the path of the soma configuration file for reference
get_soma_conf_file()

'/home/ritaank/Documents/dev/motion_synthesis/soma-root-2/soma/support_data/conf/soma_run_conf.yaml'

### Args for run_soma_on_multiple_settings:
    soma_expr_ids: list of soma experiment ids
    soma_mocap_target_ds_names: target dataset names, these should be available at mocap_base_dir
    soma_data_ids: data ids of some experiments
    tracklet_labeling_options: whether to use tracklet labeling
    ds_name_gt: gt mocap data for labeling evaluation
    soma_cfg: overloading soma_run_conf.yaml
    mosh_cfg: overloading moshpp_conf.yaml inside mosh code
    render_cfg: overload render_conf.yaml
    eval_label_cfg: eval_label.yaml
    parallel_cfg: relevant for use on IS cluster
    fast_dev_run: if True will run for a limited number of mocaps
    run_tasks: a selection of ['soma', 'mosh', 'render', 'eval_label']
    mocap_ext: file extension of the source mocap point clouds
    mosh_stagei_perseq: if True stage-i of mosh will run for every sequence instead of every subject
    fname_filter: List of strings to filter the source mocaps
    mocap_base_dir: base directory for source mocaps
    gt_mosh_base_dir: directory holding mosh results of the gt mocaps, used for v2v evaluation
    soma_work_base_dir: base directory for soma data. this directory holds: data, training_experiments, support_data
    **kwargs:

In [ ]:
soma_work_base_dir = osp.join(os.getcwd(), 'soma-root')
support_base_dir = osp.join(soma_work_base_dir, 'support_files')

mocap_base_dir = osp.join(support_base_dir, 'evaluation_mocaps/original')
timestr = time.strftime("%Y%m%d-%H%M%S")
soma_expr_id = 'V48_02_SuperSet' # the model we are using

# Settings on the trained model, including error tolerances etc.
# 5 occlusions. 3 ghost points. 0.0% real data, 100.0% synthetic data
soma_data_ids = ['OC_05_G_03_real_000_synt_100']

#### Data folders
By default, we are running on the `soma_unlabeled_mpc` folder. In this folder, we have the following structure: each subject in the dataset has a subfolder -- this becomes important when trying to MoSH the data once labeled. In this subfolder, we have the unlabeled `.c3d` files for the mocaps, as well as a `settings.json` file whose contents are simply as follows:
```json
{
	"gender":"male" // or "female"
}

```
By replicating this structure, you can run SOMA on your own mocap data. You can even process multiple folders at a time by adding them to the `target_ds_names` list below.

In [18]:
# List each of the folders you want to label
# Verify that the location of these folders is under evaluation_mocaps/original with the settings.json included
# Run on 'immersion_test' to see sample results on a few mocaps
soma_mocap_target_ds_name = 'fsp01'
# The name of the subject, for folder naming purposes
# Not necessary to change; the outputs will be in a folder with this name + the time
# subject_name = 'subjectA'

In [8]:
def select(fname):
    '''
    A filter can be applied if only a subset of mocaps is needed.
    The filter will take in a filename and return True if we want to label it.
    Filenames that will be passed in come from target_ds_names.
    '''
    # add filters here if needed
    if 'se001_EnGarde_001_engarde_LH_action31.c3d' in fname:
        return True
    return False  # subject_name: 'subjectA'

In [10]:
run_soma_on_multiple_settings(
        soma_expr_ids=[
            soma_expr_id,
        ],
        soma_mocap_target_ds_names= [soma_mocap_target_ds_name],
        soma_data_ids=soma_data_ids,
        soma_cfg={
            # 'mocap.subject_name' : f'{subject_name}_{timestr}',
            'soma.batch_size': 256,
            'dirs.support_base_dir':support_base_dir,
            'mocap.unit': 'mm',
            'save_c3d': True,
            # 'keep_nan_points': True,
            'remove_zero_trajectories': True,
            # 'mosh_stagei_perseq': True,
        },
        parallel_cfg={
            # 'max_num_jobs': 1,
            'randomly_run_jobs': True,
        },
        run_tasks=[
            'soma',
        ],
        mocap_base_dir = mocap_base_dir,
        soma_work_base_dir=soma_work_base_dir,
        mocap_ext='.c3d',
        filter_fn=select
    )

run_soma_multiple:run_soma_on_multiple_settings:135 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- fsp01 -- 20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- Found 1 mocaps for fsp01
run_soma_multiple:run_soma_on_multiple_settings:136 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- fsp01 -- 20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- mocap_fnames = ['/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/evaluation_mocaps/original/fsp01/20220701_01_001/se001_EnGarde_001_engarde_LH_action31.c3d']
run_soma_multiple:run_soma_on_multiple_settings:249 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- fsp01 -- 20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- Submitting SOMA jobs.
parallel_tools:run_parallel_jobs:54 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- fsp01 -- 20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- #Job(s) submitted: 1
parallel_tools:run_parallel_jobs:67 -- V48_02_SuperSet -- OC_05_G_03_re

100%|██████████| 2/2 [00:00<00:00,  5.38it/s]

soma_processor:apply_soma:234 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- fsp01 -- 20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- SOMA was performing at inf Hz.


soma_processor:label_tracklets:388 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- fsp01 -- 20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- 41 labels detected
soma_processor:label_tracklets:395 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- fsp01 -- 20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- Tracklet labeling yielded 100.00% non-zero mocap.
soma_processor:run_soma_once:446 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- fsp01 -- 20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- Created /home/ritaank/Documents/dev/SOMA-interface/soma-root/training_experiments/V48_02_SuperSet/OC_05_G_03_real_000_synt_100/evaluations/soma_labeled_mocap_tracklet/fsp01/20220701_01_001/se001_EnGarde_001_engarde_LH_action31.pkl
soma_processor:run_soma_once:455 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- fsp01 -- 20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- Created /home/ritaank/Documents/dev/SOMA-interface/soma-root/training_ex

## Running MoSH from SOMA
For full explanations of running MoSH, see `mosh_and_render.ipynb`
Here we simply add code to MoSH the SOMA labelling run that has been done above. It's easy right here since we have the environment and vairables all set up already.

In [22]:
type(soma_expr_id)

tuple

In [24]:
mocap_dir = osp.join(soma_work_base_dir,
                        'training_experiments',
                        soma_expr_id, soma_data_ids[0],
                        'evaluations',
                        'soma_labeled_mocap_tracklet',
                        soma_mocap_target_ds_name)

run_soma_on_multiple_settings(
    soma_expr_ids=[
        soma_expr_id,
    ],
    soma_mocap_target_ds_names=[
        'fsp01',
    ],
    soma_data_ids=
    soma_data_ids,
    mosh_cfg={
        'moshpp.verbosity': 1,  # set to two to visualize the process in psbody.mesh.mesh_viewer

        'dirs.support_base_dir': support_base_dir,
    },
    mocap_base_dir=mocap_base_dir,
    run_tasks=['mosh'],
    # fname_filter=[subject_name],
    #         fast_dev_run=True,
    mocap_ext='.c3d',
    soma_work_base_dir=soma_work_base_dir,
    parallel_cfg={
        # 'max_num_jobs': 1,  # comment to run on all mocaps
        'randomly_run_jobs': True,
    },

)

UnsupportedInterpolationType: Unsupported interpolation type resolve_mocap_subject
    full_key: dirs.eval_pkl_out_fname
    object_type=dict